In [248]:
import sys
print("Python Version:", sys.version)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
pd.set_option('display.max_rows', 500)
%matplotlib inline

Python Version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [249]:
# yymmdd format
dates = ['190803',
        '190810',
        '190817',
        '190824',
        '190831',
        '190907',
        '190914',
        '190921',
        '190928']

# iterate through dates and pull in csv and cat dataframes together
df_turns = []
for date in dates:
    df_turns.append(pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+date+'.txt'))
    
# Concat all dataframes
df_turns = pd.concat(df_turns)  

In [250]:
#Clean up the column Names
new_col_names = [name.strip() for name in df_turns.columns]
df_turns.columns = new_col_names


In [251]:
# Remove audited rows
df_turns=df_turns[df_turns.DESC!='RECOVR AUD']
# remove column?
del df_turns['DESC']

In [252]:
# Create a DateTime column that can be used for timeseries
df_turns.loc[:,'DATETIME'] = pd.to_datetime(df_turns['DATE'] + ' ' + df_turns['TIME'])

#Convert Date column to proper date type.
df_turns['DATE']  = pd.to_datetime(df_turns['DATE'],format='%m/%d/%Y')

#convert time to datetime type
df_turns['TIME']=pd.to_datetime(df_turns['TIME'], format='%H:%M:%S')

In [253]:
#backup
df_backup=df_turns.copy()

In [254]:
#Sort in order of datetime
df_turns = df_turns.sort_values(by=['DATETIME']).reset_index()

In [255]:
# Caluclate Entries Aggregated at station level by DateTime
df_turns['Entry_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

# Caluclate Exits Aggregated at station level by DateTime
df_turns['Exit_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['EXITS'].transform(pd.Series.diff)['EXITS']



In [256]:
#Absolute Value to deal with counting backward issues 
df_turns['Entry_Diff'] = abs(df_turns['Entry_Diff'])
df_turns['Exit_Diff']=abs(df_turns['Exit_Diff'])

# Calculate both
df_turns['Total_Traffic']=df_turns['Entry_Diff']+df_turns['Exit_Diff']

### Rami's Filtering

In [257]:
def Add_Weekday(data_frame, column='Date'):
    dmap = {0:'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    #Convert column to date
    data_frame[column] = pd.to_datetime(data_frame[column])
    #Add column which shows the Weekday in integer
    data_frame['Day_Number'] = data_frame[column].apply(lambda x: x.dayofweek)
    #Add column which shows the Weekday in words
    data_frame['Weekday'] = data_frame['Day_Number'].map(dmap)
    return data_frame
df_turns_backup2 = df_turns.copy()

In [258]:
df_turns = Add_Weekday(df_turns, 'DATE')

In [259]:
#Filter out the Null Values
df_turns.dropna(subset=["Entry_Diff","Exit_Diff",'Total_Traffic'], axis=0, inplace=True)
df_turns.head()

,index,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,ENTRIES,EXITS,DATETIME,Entry_Diff,Exit_Diff,Total_Traffic,Day_Number,Weekday
2554,118307,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:24:57,65,30,2019-07-27 00:24:57,2.0,0.0,2.0,5,Sat
2592,118308,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:38:57,65,30,2019-07-27 00:38:57,0.0,0.0,0.0,5,Sat
2636,118309,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:52:57,65,30,2019-07-27 00:52:57,0.0,0.0,0.0,5,Sat
2640,118310,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 00:56:25,4043,3513,2019-07-27 00:56:25,3978.0,3483.0,7461.0,5,Sat
4489,118311,PTH03,R552,00-01-08,JOURNAL SQUARE,1,PTH,2019-07-27,1900-01-01 01:06:57,65,30,2019-07-27 01:06:57,3978.0,3483.0,7461.0,5,Sat


In [260]:
dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()

C:\Users\atag3\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [261]:
#The upper_quant_df outputs a dataframe of the upper 95% values for each station
upper_quant_df = dfc_grouped.groupby(['STATION'])['Total_Traffic'].quantile([.95]).reset_index()
#Then I merge it with the grouped data 
df_turns_no_outlier = dfc_grouped.merge(upper_quant_df, on='STATION')
df_turns_no_outlier.head()

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y
0,2019-07-27,1 AV,Sat,4308.0,5146.0,9454.0,0.95,40228.0
1,2019-07-28,1 AV,Sun,4106.0,5309.0,9415.0,0.95,40228.0
2,2019-07-29,1 AV,Mon,14483.0,16525.0,31008.0,0.95,40228.0
3,2019-07-30,1 AV,Tue,16375.0,18206.0,34581.0,0.95,40228.0
4,2019-07-31,1 AV,Wed,16812.0,19777.0,36589.0,0.95,40228.0


In [262]:
#delete the rows where The difference between Total_Traffic_y and Total_Traffic_x < 0
df_turns_no_outlier['Diff'] = df_turns_no_outlier['Total_Traffic_y'] - df_turns_no_outlier['Total_Traffic_x']
df_turns_no_outlier = df_turns_no_outlier[df_turns_no_outlier['Diff']>= 0]

In [263]:
df_turns_no_outlier.sample(50)

,DATE,STATION,Weekday,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y,Diff
9798,2019-09-04,BOROUGH HALL,Wed,37462.0,25401.0,62863.0,0.95,67595.7,4732.7
12057,2019-08-27,CORTLANDT ST,Tue,20907.0,16498.0,37405.0,0.95,41184.1,3779.1
17545,2019-09-06,METS-WILLETS PT,Fri,15446.0,13973.0,29419.0,0.95,59616.5,30197.5
22946,2019-08-24,WEST FARMS SQ,Sat,4022.0,2976.0,6998.0,0.95,12708.5,5710.5
20918,2019-08-12,ROOSEVELT ISLND,Mon,7278.0,6806.0,14084.0,0.95,16047.5,1963.5
13601,2019-07-27,FLUSHING-MAIN,Sat,34218.0,29495.0,63713.0,0.95,116819.3,53106.3
12687,2019-08-27,DYCKMAN ST,Tue,12719.0,4009.0,16728.0,0.95,19080.4,2352.4
6061,2019-08-12,71 ST,Mon,5085.0,1649.0,6734.0,0.95,7760.1,1026.1
4787,2019-09-27,5 AV/53 ST,Fri,26714.0,29397.0,56111.0,0.95,61992.8,5881.8
4889,2019-09-03,5 AVE,Tue,12439.0,11200.0,23639.0,0.95,26735.9,3096.9


In [269]:
df_per_station = df_turns_no_outlier.groupby(['STATION']).sum()
df_per_station.sort_values('Total_Traffic_x', ascending=False).head(100)

,Entry_Diff,Exit_Diff,Total_Traffic_x,level_1,Total_Traffic_y,Diff
STATION,,,,,,
GRD CNTRL-42 ST,9364281.0,8661910.0,18026191.0,56.05,1.363166e+08,1.182904e+08
34 ST-PENN STA,8076349.0,7488512.0,15564861.0,56.05,1.998984e+07,4.424976e+06
34 ST-HERALD SQ,6227395.0,5999684.0,12227079.0,56.05,1.487605e+07,2.648969e+06
TIMES SQ-42 ST,5126196.0,4840524.0,9966720.0,56.05,1.236377e+07,2.397054e+06
23 ST,5597789.0,4189130.0,9786919.0,56.05,1.385681e+07,4.069892e+06
14 ST-UNION SQ,4989066.0,4766672.0,9755738.0,56.05,1.249718e+07,2.741441e+06
42 ST-PORT AUTH,5298885.0,4416285.0,9715170.0,56.05,1.172872e+07,2.013552e+06
FULTON ST,4947473.0,4437813.0,9385286.0,56.05,2.993878e+07,2.055350e+07
86 ST,4095889.0,3947739.0,8043628.0,56.05,1.077508e+07,2.731448e+06


# End of group data cleaning 

## Begin Working on Demographics info

##### Rest of the team should use the above code so we are all working with the same data.

In [170]:
# bring in Census data
cen_dat = pd.read_csv('geocoded_cen_dat.csv')
cen_vars=['STATION',
          'LAT',
          'LON',
          'p_f_pop',
          'p_emp_info',
          'p_emp_prof',
          'p_75_100k',
          'p_over_100k']

df_per_station = pd.merge(df_per_station, cen_dat[cen_vars],  how='left', left_on=['STATION'], right_on = ['STATION'])
df_per_station =df_per_station.drop_duplicates(['STATION'])
df_per_station.sort_values('Total_Traffic', ascending=False).head(100)

,STATION,index,ENTRIES,EXITS,Entry_Diff,Exit_Diff,Total_Traffic,LAT,LON,p_f_pop,p_emp_info,p_emp_prof,p_75_100k,p_over_100k
81,34 ST-PENN STA,3479245760,1479400286482,1559420280932,8764818.0,7909875.0,16674693.0,40.751056,-73.990310,0.379121,0.235714,0.228571,0.128,0.684
290,GRD CNTRL-42 ST,4017873433,953423649849,1200458232490,7456062.0,6412734.0,13868796.0,40.755861,-73.987061,0.245455,0.092105,0.355263,0.453,0.297
78,34 ST-HERALD SQ,1457426633,1758082349624,2201334016185,6276407.0,5776616.0,12053023.0,40.750087,-73.988127,0.379121,0.235714,0.228571,0.128,0.684
420,TIMES SQ-42 ST,2236642142,2231765136098,1707653583802,5517703.0,5177938.0,10695641.0,40.755861,-73.987061,0.245455,0.092105,0.355263,0.453,0.297
62,23 ST,2640537022,2151398208478,1872076315188,6117809.0,4565693.0,10683502.0,40.741201,-73.989034,0.525245,0.092991,0.238662,0.130,0.720
22,14 ST-UNION SQ,868057760,101742920593,70013494732,5386141.0,5146698.0,10532839.0,40.734537,-73.990324,0.512696,0.095516,0.175355,0.083,0.684
90,42 ST-PORT AUTH,997144338,2774818832001,2329855798997,5666984.0,4104082.0,9771066.0,40.757835,-73.991463,0.403017,0.052671,0.151951,0.140,0.485
280,FULTON ST,3249623838,1812277231584,1769638934714,5112648.0,4517619.0,9630267.0,40.710648,-74.009296,0.560466,0.042380,0.311535,0.160,0.592
140,86 ST,2635828822,601651417661,886647818951,4410643.0,4168650.0,8579293.0,40.777741,-73.951567,0.486410,0.060634,0.172976,0.146,0.520
14,125 ST,1962814981,2547950634688,1582220847721,4285348.0,3642368.0,7927716.0,40.804386,-73.937429,0.462644,0.068702,0.074046,0.058,0.072


In [247]:
cen_dat.head()

,STATION,DIVISION,all,LAT,LON,geo_type,State_FIPS,County_FIPS,Tract_FIPS,f_pop,all_pop,p_emp_info,p_f_pop,p_emp_prof,p_75_100k,p_over_100k
0,1 AV,BMT,19409616052,40.731327,-73.982435,establishment,36,61,4400,9661.0,16684.0,0.077069,0.579058,0.226378,0.166,0.396
1,103 ST,IND,2344428272,40.796286,-73.961648,establishment,36,61,18900,6869.0,11800.0,0.057668,0.582119,0.170079,0.050,0.235
2,103 ST,IRT,4382729249,40.796286,-73.961648,establishment,36,61,18900,6869.0,11800.0,0.057668,0.582119,0.170079,0.050,0.235
3,103 ST-CORONA,IRT,2641087500,40.749827,-73.862746,establishment,36,81,40501,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,104 ST,BMT,18924221,40.695146,-73.844305,establishment,36,81,2800,1602.0,3382.0,0.033613,0.473684,0.039016,0.077,0.080


In [171]:
df_per_station['Traffic_Rank'] = df_per_station['Total_Traffic'].rank(ascending=True,pct=True)*100
df_per_station['p_f_pop_Rank'] = df_per_station['p_f_pop'].rank(ascending=True,pct=True)
df_per_station['p_emp_info_Rank'] = df_per_station['p_emp_info'].rank(ascending=True,pct=True)
df_per_station['p_emp_prof_Rank'] = df_per_station['p_emp_prof'].rank(ascending=True,pct=True)
df_per_station['p_75_100k_Rank'] = df_per_station['p_75_100k'].rank(ascending=True,pct=True)
df_per_station['p_over_100k_Rank'] = df_per_station['p_over_100k'].rank(ascending=True,pct=True)

df_per_station.sort_values('Total_Traffic', ascending=False).head(100)

,STATION,index,ENTRIES,EXITS,Entry_Diff,Exit_Diff,Total_Traffic,LAT,LON,p_f_pop,p_emp_info,p_emp_prof,p_75_100k,p_over_100k,Traffic_Rank,p_f_pop_Rank,p_emp_info_Rank,p_emp_prof_Rank,p_75_100k_Rank,p_over_100k_Rank
81,34 ST-PENN STA,3479245760,1479400286482,1559420280932,8764818.0,7909875.0,16674693.0,40.751056,-73.990310,0.379121,0.235714,0.228571,0.128,0.684,100.000000,0.024927,0.998525,0.862832,0.658892,0.948980
290,GRD CNTRL-42 ST,4017873433,953423649849,1200458232490,7456062.0,6412734.0,13868796.0,40.755861,-73.987061,0.245455,0.092105,0.355263,0.453,0.297,99.735450,0.004399,0.877581,0.998525,0.998542,0.712828
78,34 ST-HERALD SQ,1457426633,1758082349624,2201334016185,6276407.0,5776616.0,12053023.0,40.750087,-73.988127,0.379121,0.235714,0.228571,0.128,0.684,99.470899,0.024927,0.998525,0.862832,0.658892,0.948980
420,TIMES SQ-42 ST,2236642142,2231765136098,1707653583802,5517703.0,5177938.0,10695641.0,40.755861,-73.987061,0.245455,0.092105,0.355263,0.453,0.297,99.206349,0.004399,0.877581,0.998525,0.998542,0.712828
62,23 ST,2640537022,2151398208478,1872076315188,6117809.0,4565693.0,10683502.0,40.741201,-73.989034,0.525245,0.092991,0.238662,0.130,0.720,98.941799,0.599707,0.886431,0.892330,0.666181,0.969388
22,14 ST-UNION SQ,868057760,101742920593,70013494732,5386141.0,5146698.0,10532839.0,40.734537,-73.990324,0.512696,0.095516,0.175355,0.083,0.684,98.677249,0.472141,0.905605,0.731563,0.376093,0.948980
90,42 ST-PORT AUTH,997144338,2774818832001,2329855798997,5666984.0,4104082.0,9771066.0,40.757835,-73.991463,0.403017,0.052671,0.151951,0.140,0.485,98.412698,0.033724,0.647493,0.685841,0.695335,0.806122
280,FULTON ST,3249623838,1812277231584,1769638934714,5112648.0,4517619.0,9630267.0,40.710648,-74.009296,0.560466,0.042380,0.311535,0.160,0.592,98.148148,0.821114,0.563422,0.973451,0.830904,0.862974
140,86 ST,2635828822,601651417661,886647818951,4410643.0,4168650.0,8579293.0,40.777741,-73.951567,0.486410,0.060634,0.172976,0.146,0.520,97.883598,0.280059,0.721239,0.724189,0.734694,0.822157
14,125 ST,1962814981,2547950634688,1582220847721,4285348.0,3642368.0,7927716.0,40.804386,-73.937429,0.462644,0.068702,0.074046,0.058,0.072,97.619048,0.148094,0.783186,0.476401,0.221574,0.327988


In [176]:
import folium
from folium.plugins import MarkerCluster

In [244]:
#reset index of df per station
df_per_station = df_per_station.sort_values('Total_Traffic', ascending=False).head(100).reset_index(drop=True)

#stations
stations = df_per_station[['LAT','LON']]
stations_list=stations.values.tolist()
#Define coordinates of map center
nyc_coords = [40.7528, -73.9560]

#Create the map
my_map = folium.Map(location = nyc_coords, zoom_start = 12)

#Display the map
my_map

for point in range(0, len(stations_list)):
    if point <=10:
        folium.Marker(stations_list[point], popup=df_per_station.STATION[point]).add_to(my_map)
    else:
        folium.Circle(stations_list[point], popup=df_per_station.STATION[point],fill=True,radius=100,color='red').add_to(my_map)
    
my_map

AttributeError: 'DataFrame' object has no attribute 'STATIOJN'